In [1]:
# TODO: Fix linear regression overflow error
# TODO: Fix linear regression convergence error

#### Libraries

In [2]:
import numpy as np
from IPython.display import clear_output
from sklearn.datasets import load_boston
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import classification_report

#### Auxiliar Function

In [3]:
def add_bias(x):
  return np.hstack((np.ones((x.shape[0], 1)), x))

#### Linear Regression

##### Training with Gradient Descent

In [4]:
def fit_linear(x, y, alpha=.01, min_cost=10**-4, max_iter=10**2):
  i = 0
  m, n = x.shape
  cost = [min_cost + 100]
  theta = np.random.rand(n, 1)
  while i < max_iter and cost[-1] > min_cost:
    i += 1
    h = np.dot(x, theta)
    # BEGIN ERROR !!!!!!!!!!!!!!!!!!!!!!!
    theta = theta - (alpha / m) * np.sum((h - y) * x)
    cost.append(np.sum(np.square(h - y)) / (2 * m))
    # END ERROR !!!!!!!!!!!!!!!!!!!!!!!!!
  return theta, cost[1:]

##### Training with Normal Equations

In [5]:
def fit_linear_wne(x, y):
  # calculate the coefficient theta = (x^(t)x)^(-1)x^(t)y
  theta = np.dot(np.linalg.pinv(np.dot(x.T, x)), np.dot(x.T, y))
  cost = mean_squared_error(y, np.dot(x, theta))
  return theta, cost

##### Execution

In [6]:
boston = load_boston()
scaler = StandardScaler()

x_train, x_test, y_train, y_test = train_test_split(
  boston.data, boston.target.reshape((-1, 1)), test_size=.2)

x_train = add_bias(scaler.fit_transform(x_train))
x_test  = add_bias(scaler.transform(x_test))

In [7]:
theta, cost = fit_linear(x_train, y_train, alpha=.1)
error = mean_squared_error(y_test, np.dot(x_test, theta))
print(f'linear regression with gradient descent MSE: {error:.3f}')

linear regression with gradient descent MSE: 276534793281.868


In [8]:
theta, cost = fit_linear_wne(x_train, y_train)
error = mean_squared_error(y_test, np.dot(x_test, theta))
print(f'linear regression with normal equations MSE: {error:.3f}')

linear regression with normal equations MSE: 17.986


#### Logistic Regression

##### Training with (Regularized) Gradient Descent

In [9]:
def sigmoid(x):
  return 1 / (1 + np.exp(-x))

In [10]:
def fit_logistic(x, y, lambd=1, alpha=.01, min_cost=10**-4, max_iter=10**4):
  i = 0
  m, n = x.shape
  epsilon = 10 ** -8
  cost = [min_cost + 100]
  theta = np.random.rand(n, 1)
  while cost[-1] > min_cost and i < max_iter:
    i += 1
    theta_aux = theta
    theta_aux[0] = 0
    h = sigmoid(np.dot(x, theta))
    grad = np.dot(x.T, h - y) / m
    grad += (lambd / m) * theta
    theta -= alpha * grad
    cost_i = y * np.log(h + epsilon)
    cost_i += (1 - y) * np.log(1 - h + epsilon)
    cost_i += (lambd / (2 * m)) * np.dot(theta_aux.T, theta_aux)
    cost.append(- np.mean(cost_i))
  return theta, cost[1:]

In [11]:
def fit_logistic_ova(x, y, lambd=1, alpha=.01, min_cost=10**-8, max_iter=10**4):
  cost = []
  labels = np.unique(y)
  theta = np.random.rand(x.shape[1], labels.shape[0])
  for i, label in enumerate(labels):
    yi = (y == label).astype(float)
    theta_i, cost_i = fit_logistic(x, yi, 
                                   lambd=lambd,
                                   alpha=alpha, 
                                   min_cost=min_cost, 
                                   max_iter=max_iter)
    cost.append(cost_i)
    theta[:, i] = theta_i.ravel()
  return theta, cost

##### Execution

In [13]:
digits = load_digits()
scaler = StandardScaler()

x_train, x_test, y_train, y_test = train_test_split(
  digits.data, digits.target.reshape((-1, 1)), test_size=.2)

x_train = add_bias(scaler.fit_transform(x_train))
x_test  = add_bias(scaler.transform(x_test))

In [16]:
theta, cost = fit_logistic_ova(x_train, y_train)
y_pred = np.argmax(sigmoid(np.dot(x_train, theta)), axis=1)
print('\t\ttraining classification report')
print(classification_report(y_train, y_pred))

		training classification report
              precision    recall  f1-score   support

           0       0.90      1.00      0.95       146
           1       0.95      0.90      0.92       143
           2       0.89      0.97      0.93       137
           3       0.94      0.91      0.92       144
           4       0.99      0.95      0.97       147
           5       0.91      0.98      0.94       148
           6       0.97      0.97      0.97       143
           7       0.95      0.98      0.96       142
           8       0.91      0.80      0.85       142
           9       0.90      0.84      0.87       145

    accuracy                           0.93      1437
   macro avg       0.93      0.93      0.93      1437
weighted avg       0.93      0.93      0.93      1437



In [15]:
y_pred = np.argmax(sigmoid(np.dot(x_test, theta)), axis=1)
print('\t\ttesting classification report')
print(classification_report(y_test, y_pred))

		testing classification report
              precision    recall  f1-score   support

           0       0.80      1.00      0.89        32
           1       0.89      0.87      0.88        39
           2       0.95      0.90      0.92        40
           3       0.93      0.97      0.95        39
           4       0.89      0.94      0.91        34
           5       0.88      0.88      0.88        34
           6       0.90      0.92      0.91        38
           7       0.85      0.95      0.90        37
           8       0.93      0.78      0.85        32
           9       0.92      0.69      0.79        35

    accuracy                           0.89       360
   macro avg       0.89      0.89      0.89       360
weighted avg       0.90      0.89      0.89       360



#### References

[1] LeCun, Y., 1998. The MNIST database of handwritten digits. http://yann.lecun.com/exdb/mnist/.

[2] Ng, A., 2017. Machine learning course. Coursera [online]. Available at: https://www.coursera.org/learn/machinelearning.